In [ ]:
import torch
from torch.utils.data import Dataset
import os
import pickle
import argparse
from collections import defaultdict
import pandas as pd
import numpy as np
import gzip
import shutil
import spacy 
import pandas as pd 
from tqdm import tqdm
from collections import Counter
import kora.install.rdkit
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Draw, MolFromSmiles, MolToSmiles

In [ ]:
checkpoint = torch.load('/content/drive/MyDrive/project_best.pth')
model.load_state_dict(checkpoint['model_state_dict'])s

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

def eval_mlp(X, y, rate, n_repeats):
    auc = np.empty(n_repeats)
    for i in range(n_repeats):
        clf = MLPClassifier(max_iter=1000)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1-rate, stratify=y)
        clf.fit(X_train, y_train)
        y_score = clf.predict_proba(X_test)
        auc[i] = roc_auc_score(y_test, y_score[:,1])
    res = {}
    res['auc mean'] = np.mean(auc)
    res['auc std'] = np.mean(np.std(auc, axis=0))
    return res

def get_inputs(sm):
    seq_len = 220
    sm = sm.split()
    if len(sm) > 218:
        sm = sm[:109]+sm[-109:]
    ids = [vocab.word_dict.get(token, unk_index) for token in sm]
    ids = [sos_index] + ids + [eos_index]
    seg = [1] * len(ids)
    padding = [pad_index] * (seq_len - len(ids))
    ids.extend(padding)
    seg.extend(padding)
    return ids, seg

def get_array(smiles):
    x_id, x_seg = [], []
    for sm in smiles:
        a,b = get_inputs(sm)
        x_id.append(a)
        x_seg.append(b)
    return torch.tensor(x_id), torch.tensor(x_seg)

In [ ]:
fs = [('/content/drive/MyDrive/BBBP.csv', 'p_np', 'smiles'), ('/content/drive/MyDrive/HIV.csv', 'HIV_active', 'smiles'), ('/content/drive/MyDrive/bace.csv', 'Class', 'mol')]
for f in fs:
    model.eval()
    pad_index = 0
    unk_index = 1
    eos_index = 2
    sos_index = 3
    mask_index = 4
    df = pd.read_csv(f[0])
    rates = 2**np.arange(7)/80
    x_split = [split(sm) for sm in df[f[2]].values]
    xid, _ = get_array(x_split)
    X = model.encode(torch.t(xid).cuda())
    mean_score = np.zeros(len(rates))
    print(X.shape)
    for i, rate in enumerate(rates):
        score_dic = eval_mlp(X, df[f[1]].values, rate, 20)
        mean_score[i] = score_dic['auc mean']
        print(rate, score_dic)
    
    print(np.mean(mean_score))